In [1]:
import getpass
import os

In [2]:
os.environ["MISTRAL_API_KEY"] = getpass.getpass()

from langchain_mistralai import ChatMistralAI

model = ChatMistralAI(model="mistral-large-latest")

 ········


In [22]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


In [32]:
os.environ["LANGCHAIN_PROJECT"] = "pr-enchanted-afoul-34"
os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"

In [4]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

result = model.invoke(messages)

In [6]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parser.invoke(result)

'Hello! Translates to "Ciao!" in Italian. So, "Hi!" would be "Ciao!" in Italian.'

In [7]:
chain = model | parser
chain.invoke(messages)

'Hello! In Italian, this is typically translated as "Ciao!" Please note that Italian, like many languages, has formal and informal ways of addressing people. "Ciao" is an informal greeting, suitable for friends and family, or in casual settings. If you\'re addressing someone in a formal context or someone you don\'t know well, "Salve" or "Buongiorno" (Good day) might be more appropriate.'

## Use Prompt Templates
User input is transformed into a list of system message and user input. Some times user input is also formatted. 
This transformation process is assisted by Langchains Prompt templates

In [8]:
from langchain.prompts import ChatPromptTemplate

In [9]:
system_template = "Translate the following into {language}"

In [13]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system",system_template),
     ("user", "{text}")
    ]
)

In [15]:
# prompt_tempalate takes dictionary as input where "language" and "text" are the keys. 
result = prompt_template.invoke({"language":"italian", "text":"Hi"})

In [16]:
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian'), HumanMessage(content='Hi')])

In [17]:
result.to_messages()

[SystemMessage(content='Translate the following into italian'),
 HumanMessage(content='Hi')]

In [33]:
#chaining togather the components with LECL (Langchain Expression Language)
chain = prompt_template | model | parser


In [34]:
result = chain.invoke({"language":"italian", "text":"Hi"})

In [31]:
result

'Hello! The translation of "Hi" into Italian is "Ciao".'

## Serving with LangServe
Now that we've built an application, we need to serve it. That's where LangServe comes in. LangServe helps developers deploy LangChain chains as a REST API. You do not need to use LangServe to use LangChain, but in this guide we'll show how you can deploy your app with LangServe.

While the first part of this guide was intended to be run in a Jupyter Notebook or script, we will now move out of that. We will be creating a Python file and then interacting with it from the command line.

Install with:

```python
pip install "langserve[all]"
```
### Server
To create a server for our application we'll make a serve.py file. This will contain our logic for serving our application. It consists of three things:

The definition of our chain that we just built above
Our FastAPI app
A definition of a route from which to serve the chain, which is done with langserve.add_routes
```python
#!/usr/bin/env python
from typing import List

from fastapi import FastAPI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langserve import add_routes

# 1. Create prompt template
system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages([
    ('system', system_template),
    ('user', '{text}')
])

# 2. Create model
model = ChatOpenAI()

# 3. Create parser
parser = StrOutputParser()

# 4. Create chain
chain = prompt_template | model | parser


# 4. App definition
app = FastAPI(
  title="LangChain Server",
  version="1.0",
  description="A simple API server using LangChain's Runnable interfaces",
)

# 5. Adding chain route

add_routes(
    app,
    chain,
    path="/chain",
)

if __name__ == "__main__":
    import uvicorn

    uvicorn.run(app, host="localhost", port=8000)
```
API Reference:ChatPromptTemplate | StrOutputParser | ChatOpenAI
And that's it! If we execute this file:
```python
python serve.py
```
we should see our chain being served at http://localhost:8000.

### Playground
Every LangServe service comes with a simple built-in UI for configuring and invoking the application with streaming output and visibility into intermediate steps. Head to http://localhost:8000/chain/playground/ to try it out! Pass in the same inputs as before - {"language": "italian", "text": "hi"} - and it should respond same as before.

### Client
Now let's set up a client for programmatically interacting with our service. We can easily do this with the [langserve.RemoteRunnable](/docs/langserve/#client). Using this, we can interact with the served chain as if it were running client-side.

```python
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "italian", "text": "hi"})
```

'Ciao'

To learn more about the many other features of LangServe head here.

### Conclusion
That's it! In this tutorial you've learned how to create your first simple LLM application. You've learned how to work with language models, how to parse their outputs, how to create a prompt template, chaining them with LCEL, how to get great observability into chains you create with LangSmith, and how to deploy them with LangServe.

This just scratches the surface of what you will want to learn to become a proficient AI Engineer. Luckily - we've got a lot of other resources!

For further reading on the core concepts of LangChain, we've got detailed Conceptual Guides.

If you have more specific questions on these concepts, check out the following sections of the how-to guides: